# **Ejercicios de pair programming Módulo 3 Sprint 1**
## **Regresión lineal: Lección 11**
### **Decision Tree**

---

In [6]:
# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Para navegar entre directorios
# ------------------------------------
import os

# Modelado y evaluación
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('once')

pd.options.display.max_columns = None

En el pair programming de hoy debéis usar el csv que guardastéis cuando hicistéis el pairprgramming de codificicación (este csv debería tener las variables estadandarizas).

In [2]:
os.getcwd()

'/mnt/c/Users/User/Desktop/ADALAB/MODULO3/promo-d-da-modulo3-sprint1-elisa-lidya/regresion-lineal'

In [4]:
os.chdir('/mnt/c/Users/User/Desktop/ADALAB/MODULO3/promo-d-da-modulo3-sprint1-elisa-lidya/regresion-lineal')

In [7]:
df_socio = pd.read_csv('../datos/esperanza_vida_transformado.csv', index_col=0)
df_socio.head(2)

,population_in_millions,population_density,urban_population_y,gdp_per_capita,health_total_expenditure,health_care_index,quantity_of_physicians,quality_of_life_index,purchasing_power_index,safety_index,pollution_index,pop_using_improved_drinking_water,life_expectancy_at_birth_total_bc,region_CentralAmerica,region_EasternAsia,region_EasternEurope,region_NorthernAfrica,region_NorthernAmerica,region_NorthernEurope,region_Oceania,region_South-easternAsia,region_SouthAmerica,region_SouthernAfrica,region_SouthernAsia,region_SouthernEurope,region_WesternAsia,region_WesternEurope
country,,,,,,,,,,,,,,,,,,,,,,,,,,,
Argentina,-0.178879,-0.273196,1.103154,-0.439011,-1.075299,0.742566,3,0.131610,-0.436682,-1.655219,0.057166,0.474870,1.596496e+11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
Australia,-0.262734,-0.283298,0.771296,1.362030,0.630170,0.837385,2,1.015861,1.027351,-0.217071,-1.285658,0.710275,2.611737e+11,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0



En pairprogramming anteriores ajustastéis vuestro datos a una regresión lineal. El objetivo de hoy es:
 - Ajustar el modelo a un Decision Tree


In [8]:
X = df_socio.drop('life_expectancy_at_birth_total_bc', axis=1)
y = df_socio['life_expectancy_at_birth_total_bc']

In [14]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 63)

arbol = DecisionTreeRegressor(random_state = 137)

arbol.fit(x_train, y_train)

max_features = np.sqrt(len(x_train.columns))
print(f'La máxima cantidad de features que podemos incluir en el modelo es {max_features}.')
print(f'La máxima profundidad que puede tener nuestro modelo es {arbol.tree_.max_depth}')

y_pred_test = arbol.predict(x_test)
y_pred_train = arbol.predict(x_train)

La máxima cantidad de features que podemos incluir en el modelo es 5.0990195135927845.
La máxima profundidad que puede tener nuestro modelo es 10


In [15]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                "set": ["test", "train"]}
    
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [16]:
resultados1 = metricas(y_test, y_train, y_pred_test, y_pred_train, 'Arbol de decisión 1')

In [17]:
resultados1

,MAE,MSE,RMSE,R2,set,modelo
0,3.997748e+10,2.839250e+21,5.328461e+10,0.122523,test,Arbol de decisión 1
1,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,train,Arbol de decisión 1


`Vemos que tenemos un claro caso de overfitting, así que utilizamos GridSearchCV para ajustar nuestro hiperparámetros.`

In [19]:
print(f'La máxima cantidad de features que podemos incluir en el modelo es {max_features}.')
print(f'La máxima profundidad que puede tener nuestro modelo es {arbol.tree_.max_depth}')

La máxima cantidad de features que podemos incluir en el modelo es 5.0990195135927845.
La máxima profundidad que puede tener nuestro modelo es 10


In [20]:
param = {"max_depth": [2,4,6],  
        "max_features": [1,2,3,4,5],        
        "min_samples_split": [10,50,100],
        "min_samples_leaf": [10,50,100]}

In [21]:
gs = GridSearchCV(
            estimator=DecisionTreeRegressor(), 
            param_grid= param, 
            cv=10, 
            verbose=-1, 
            return_train_score = True, 
            scoring="neg_mean_squared_error")

In [22]:
gs.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [2, 4, 6],
                         'max_features': [1, 2, 3, 4, 5],
                         'min_samples_leaf': [10, 50, 100],
                         'min_samples_split': [10, 50, 100]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [23]:
modelo1 = gs.best_estimator_
modelo1

DecisionTreeRegressor(max_depth=2, max_features=5, min_samples_leaf=10,
                      min_samples_split=10)

In [25]:
param2 = {"max_depth": [1,2,3],  
        "max_features": [1,2,3,4,5],        
        "min_samples_split": [5,10,15,25],
        "min_samples_leaf": [5,10,15,25]}

In [27]:
gs2 = GridSearchCV(
            estimator=DecisionTreeRegressor(), 
            param_grid= param2, 
            cv=10, 
            verbose=-1, 
            return_train_score = True, 
            scoring="neg_mean_squared_error")

gs2.fit(x_train, y_train)

modelo2 = gs2.best_estimator_
modelo2

DecisionTreeRegressor(max_depth=2, max_features=3, min_samples_leaf=5,
                      min_samples_split=10)

In [ ]:
#volvemos a hacer el método
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 63)

arbol = DecisionTreeRegressor(random_state = 137)

arbol.fit(x_train, y_train)

max_features = np.sqrt(len(x_train.columns))
print(f'La máxima cantidad de features que podemos incluir en el modelo es {max_features}.')
print(f'La máxima profundidad que puede tener nuestro modelo es {arbol.tree_.max_depth}')

y_pred_test = arbol.predict(x_test)
y_pred_train = arbol.predict(x_train)

 - Extraer las métricas


 - Debatid entre vosotras que modelo es mejor y por qué (basándose en las métricas)